In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import re
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
import string
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from keras.models import Model
from keras.layers import LSTM, Activation, Dense, Dropout, Input, Embedding, SpatialDropout1D
from keras.optimizers import RMSprop
from keras.preprocessing import sequence
from keras.utils import to_categorical
from keras.callbacks import EarlyStopping, ModelCheckpoint
from keras.models import Sequential
from keras.utils import pad_sequences
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [2]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/YoutubeCommentsDataSet.csv


In [3]:
df=pd.read_csv("/kaggle/input/YoutubeCommentsDataSet.csv")

In [4]:
df.head()

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive


In [5]:
df.shape

(18408, 2)

# **remove null**

In [6]:
df.isnull().sum()

Comment      44
Sentiment     0
dtype: int64

In [7]:
df.dropna(inplace=True)

In [8]:
df.isnull().sum()

Comment      0
Sentiment    0
dtype: int64

# **check dublicates**

In [9]:
df.duplicated().sum()

490

In [10]:
df.drop_duplicates(inplace=True)

In [11]:
df.duplicated().sum()

0

In [12]:
df['Sentiment'].value_counts()

Sentiment
positive    11054
neutral      4503
negative     2317
Name: count, dtype: int64

# **lower case**

In [13]:
df['Comment'][3]

'whenever i go to a place that doesn’t take apple pay doesn’t happen too often it’s such a drag between ‘contactless covid’ habits and my getting the apple card i’ve gotten so used to apple pay that i get seriously annoyed when a store doesn’t take it it feels like a shock it’s crazy how quickly it took over my shopping routine i’ve officially been brainwashed by apple because now it feels so inconvenient to even carry a physical card in my pocket'

In [14]:
df['Comment'] = df['Comment'].str.lower()

In [15]:
df

,Comment,Sentiment
0,lets not forget that apple pay in 2014 require...,neutral
1,here in nz 50 of retailers don’t even have con...,negative
2,i will forever acknowledge this channel with t...,positive
3,whenever i go to a place that doesn’t take app...,negative
4,apple pay is so convenient secure and easy to ...,positive
...,...,...
18403,i really like the point about engineering tool...,positive
18404,i’ve just started exploring this field and thi...,positive
18405,excelente video con una pregunta filosófica pr...,neutral
18406,hey daniel just discovered your channel a coup...,positive


# **remove_html_tags**

In [16]:
import re
def remove_html_tags(text):
    pattern = re.compile('<.*?>')
    return pattern.sub(r'', text)

In [17]:
text = "<html><body><p> Movie 1</p><p> Actor - Aamir Khan</p><p> Click here to <a href='http://google.com'>download</a></p></body></html>"


In [18]:
remove_html_tags(text)

' Movie 1 Actor - Aamir Khan Click here to download'

In [19]:
df['Comment'] = df['Comment'].apply(remove_html_tags)

# **punctuation handling**

In [20]:
import string,time
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [21]:
exclude = string.punctuation
exclude

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [22]:
text = 'string. With. Punctuation?'

In [23]:

def remove_punc1(text):
    return text.translate(str.maketrans('', '', exclude))

In [24]:
remove_punc1(text)

'string With Punctuation'

In [25]:
df['Comment']=df['Comment'].apply(remove_punc1)

In [26]:
df.sample(4)

,Comment,Sentiment
10647,can’t wait for the new mix to drop i can feel ...,positive
5721,i like that it looks like a totally original r...,positive
4991,c’est le mercredi 7 septembre pas mardi,positive
1594,i absolutely loved baseball in wii so i hope i...,positive


# **To remove comments in languages other than a specific one (e.g., English or Portuguese), you can use langdetect to detect and filter them.**

In [27]:
!pip install langdetect  # Install langdetect (only once)

from langdetect import detect

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 981.5/981.5 kB 14.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for langdetect: filename=langdetect-1.0.9-py3-none-any.whl size=993222 sha256=579c02b6b3c9f81b8d6de53ae6f2457d83b52dc17b02d99f948b6b54758c9f41
  Stored in directory: /root/.cache/pip/wheels/95/03/7d/59ea870c70ce4e5a370638b5462a7711ab78fba2f655d05106
Successfully built langdetect


In [28]:
def detect_language(text):
    try:
        return detect(text)
    except:
        return "unknown"  # Handle errors for empty or unrecognized text


In [29]:
df["language"] = df["Comment"].apply(detect_language)  # Detect language
df = df[df["language"].isin(["en"])]  # Keep only selected languages
df.drop(columns=["language"], inplace=True)  # Remove extra column if not needed


<ipython-input-29-a09efb846df4>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.drop(columns=["language"], inplace=True)  # Remove extra column if not needed


In [30]:
df.sample(4)

,Comment,Sentiment
9746,hello dona your food always looks delicious,positive
10385,we have s400 and other top class defense syste...,neutral
6693,as one vigilante once remarked when you want t...,neutral
9300,hands down gabriella is one of if not the best...,positive


In [31]:
print(df.iloc[1630])  # Indexing starts from 0, so use 163

Comment      i am almost at my 20th hour of watching tedx t...
Sentiment                                             positive
Name: 1911, dtype: object


In [32]:
df.shape

(15882, 2)

# **stopwords**

In [33]:
from nltk.corpus import stopwords
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [34]:
stopwords.words('english')

['a',
 'about',
 'above',
 'after',
 'again',
 'against',
 'ain',
 'all',
 'am',
 'an',
 'and',
 'any',
 'are',
 'aren',
 "aren't",
 'as',
 'at',
 'be',
 'because',
 'been',
 'before',
 'being',
 'below',
 'between',
 'both',
 'but',
 'by',
 'can',
 'couldn',
 "couldn't",
 'd',
 'did',
 'didn',
 "didn't",
 'do',
 'does',
 'doesn',
 "doesn't",
 'doing',
 'don',
 "don't",
 'down',
 'during',
 'each',
 'few',
 'for',
 'from',
 'further',
 'had',
 'hadn',
 "hadn't",
 'has',
 'hasn',
 "hasn't",
 'have',
 'haven',
 "haven't",
 'having',
 'he',
 "he'd",
 "he'll",
 'her',
 'here',
 'hers',
 'herself',
 "he's",
 'him',
 'himself',
 'his',
 'how',
 'i',
 "i'd",
 'if',
 "i'll",
 "i'm",
 'in',
 'into',
 'is',
 'isn',
 "isn't",
 'it',
 "it'd",
 "it'll",
 "it's",
 'its',
 'itself',
 "i've",
 'just',
 'll',
 'm',
 'ma',
 'me',
 'mightn',
 "mightn't",
 'more',
 'most',
 'mustn',
 "mustn't",
 'my',
 'myself',
 'needn',
 "needn't",
 'no',
 'nor',
 'not',
 'now',
 'o',
 'of',
 'off',
 'on',
 'once',
 'on

In [35]:
def remove_stopwords(text):
    new_text = []

    for word in text.split():
        if word in stopwords.words('english'):
            new_text.append('')
        else:
            new_text.append(word)
    x = new_text[:]
    new_text.clear()
    return " ".join(x)

In [36]:

df['Comment'].apply(remove_stopwords)

0        lets  forget  apple pay  2014 required  brand ...
1          nz 50  retailers don’t even  contactless cre...
2          forever acknowledge  channel   help   lesson...
3        whenever  go   place  doesn’t take apple pay d...
4        apple pay   convenient secure  easy  use  used...
                               ...                        
18402    back    learning  code  didn’t know anything  ...
18403     really like  point  engineering toolboxes  th...
18404    i’ve  started exploring  field     really good...
18406    hey daniel  discovered  channel  couple days a...
18407      great focus  key  playful approach  also spe...
Name: Comment, Length: 15882, dtype: object

# **Target Variable Splitting¶**

In [37]:
# Split data to be used in the models
# Create matrix of features
x = df.drop('Sentiment', axis = 1) # grabs everything else but 'Price'

# Create target variable
y = df['Sentiment'] # y is the column we're trying to predict

# **label encoder¶**

In [38]:
# import labelencoder from sklearn
from sklearn.preprocessing import LabelEncoder

In [39]:
# Encode the target variable
encoder = LabelEncoder()
y = encoder.fit_transform(y)

In [40]:
y

array([1, 0, 2, ..., 2, 2, 2])

In [41]:
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test = train_test_split(x,y,test_size=0.2,random_state=1)

In [42]:

x_train.shape

(12705, 1)

In [43]:
x_test.shape

(3177, 1)

# **FEATURE EXTRACTION**

# **BOW**

In [44]:

# Applying BoW
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer()

In [45]:

x_train_bow = cv.fit_transform(x_train['Comment']).toarray()
x_test_bow = cv.transform(x_test['Comment']).toarray()


In [46]:

x_train_bow

array([[0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       ...,
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0],
       [0, 0, 0, ..., 0, 0, 0]])

In [47]:
y_train

array([2, 2, 1, ..., 2, 2, 2])

In [48]:
from sklearn.ensemble import RandomForestClassifier
print('RandomForest Classifier ')
rf = RandomForestClassifier()

rf.fit(x_train_bow,y_train)


RandomForest Classifier 


RandomForestClassifier()

In [49]:
from sklearn.metrics import accuracy_score
# prediction from the model
y_pred = rf.predict(x_test_bow)
# Score It

print('RandomForest Classifier')
# Accuracy
print('--'*30)
rf_accuracy = round(accuracy_score(y_test, y_pred) * 100,2)
print('Accuracy', rf_accuracy,'%')

RandomForest Classifier
------------------------------------------------------------
Accuracy 65.56 %


# **N grams**

In [50]:
cv = CountVectorizer(ngram_range=(1,2))


x_train_bow = cv.fit_transform(x_train['Comment']).toarray()
x_test_bow = cv.transform(x_test['Comment']).toarray()

In [51]:
from sklearn.ensemble import RandomForestClassifier
print('RandomForest Classifier ')
rf = RandomForestClassifier()

rf.fit(x_train_bow,y_train)

RandomForest Classifier 


RandomForestClassifier()

In [52]:
from sklearn.metrics import accuracy_score
# prediction from the model
y_pred = rf.predict(x_test_bow)
# Score It

print('RandomForest Classifier')
# Accuracy
print('--'*30)
rf_accuracy = round(accuracy_score(y_test, y_pred) * 100,2)
print('Accuracy', rf_accuracy,'%')

RandomForest Classifier
------------------------------------------------------------
Accuracy 64.12 %


# **TFIDF**

In [53]:

from sklearn.feature_extraction.text import TfidfVectorizer
tfidf = TfidfVectorizer()

In [54]:
x_train_tfidf = tfidf.fit_transform(x_train['Comment']).toarray()
x_test_tfidf = tfidf.transform(x_test['Comment'])

In [55]:
from sklearn.ensemble import RandomForestClassifier
print('RandomForest Classifier ')
rf = RandomForestClassifier()

rf.fit(x_train_tfidf,y_train)

RandomForest Classifier 


RandomForestClassifier()

In [56]:
from sklearn.metrics import accuracy_score
# prediction from the model
y_pred = rf.predict(x_test_tfidf)
# Score It

print('RandomForest Classifier')
# Accuracy
print('--'*30)
rf_accuracy = round(accuracy_score(y_test, y_pred) * 100,2)
print('Accuracy', rf_accuracy,'%')

RandomForest Classifier
------------------------------------------------------------
Accuracy 65.38 %
